In [17]:
# Importar depedencias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, silhouette_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans

In [2]:
# Cargar los datos
ruta_archivo = 'data/df_mexico_conectado_con_indice_marginacion.csv'
datos = pd.read_csv(ruta_archivo)

In [5]:
# Función para preprocesar los datos
def preprocesar_datos(df):
    # Identificar columnas numéricas y categóricas
    columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    columnas_categoricas = df.select_dtypes(include=['object']).columns
    # Crear transformadores para cada tipo de columna
    transformador_numerico = Pipeline(steps=[ ('scaler', StandardScaler()) ])
    transformador_categorico = Pipeline(steps=[ ('onehot', OneHotEncoder(handle_unknown='ignore')) ])
    # Crear y aplicar el transformador de columnas
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', transformador_numerico, columnas_numericas),
            ('cat', transformador_categorico, columnas_categoricas)
        ]
    )
    # Ajustar y transformar los datos
    df_preprocesado = preprocessor.fit_transform(df)
    return df_preprocesado
# Aplicar preprocesar datos y almacenarlos en una variable para su futuro uso
datos_preprocesados = preprocesar_datos(datos)

In [7]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    datos_preprocesados, 
    datos['grado_marginacion_2020'], # 'grado_marginacion_2020' es la variable objetivo
    test_size=0.2, 
    random_state=42
)

# Definir modelos de aprendizaje automático
modelos = {
    "SVM": SVC(),
    "Árbol de Decisión": DecisionTreeClassifier(),
    "K-Means": KMeans(n_clusters=3) # Se puede ajustar el # de clústers
}

In [9]:
# Función para entrenar y evaluar modelos SVM, Árbol de Decisión (Supervisados)
def entrenar_evaluar_modelo(modelo, X_train, X_test, y_train, y_test):
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_test)
    print(classification_report(y_test, predicciones))
    return accuracy_score(y_test, predicciones)

# Función para entrenar y evaluar el modelo K-Means (No Supervisado)
def entrenar_evaluar_kmeans(modelo, X):
    modelo.fit(X)
    etiquetas = modelo.labels_
    score = silhouette_score(X, etiquetas)
    return score

In [33]:
# Aplicar Entrenar y evaluar modelos
resultados = {}
for nombre, modelo in modelos.items():
    print(f"Evaluando modelo: {nombre}")
    if nombre == "K-Means":  # Proceso especial para K-Means u otros algoritmos no supervisados
        modelo_kmeans = KMeans(n_clusters=3, n_init='auto')  # Ajustar el número de clusters
        score_kmeans = entrenar_evaluar_kmeans(modelo_kmeans, X_train)
        print(f"Silhouette Score para K-Means: {score_kmeans}")
        resultados[nombre] = score_kmeans
        continue
    else:
        resultados[nombre] = entrenar_evaluar_modelo(modelo, X_train, X_test, y_train, y_test)

Evaluando modelo: SVM
              precision    recall  f1-score   support

        Alto       1.00      1.00      1.00       124
        Bajo       0.99      0.99      0.99       110
       Medio       1.00      0.99      0.99        97
    Muy alto       0.97      1.00      0.99        35
    Muy bajo       0.99      0.99      0.99       128

    accuracy                           0.99       494
   macro avg       0.99      0.99      0.99       494
weighted avg       0.99      0.99      0.99       494

Evaluando modelo: Árbol de Decisión
              precision    recall  f1-score   support

        Alto       1.00      1.00      1.00       124
        Bajo       1.00      0.99      1.00       110
       Medio       0.99      1.00      0.99        97
    Muy alto       1.00      1.00      1.00        35
    Muy bajo       1.00      1.00      1.00       128

    accuracy                           1.00       494
   macro avg       1.00      1.00      1.00       494
weighted avg       

In [23]:
# Imprimir resultados y seleccionar el mejor modelo
mejor_modelo = max(resultados, key=resultados.get)
print(f"El mejor modelo es: {mejor_modelo} con una precisión de {resultados[mejor_modelo]}")


El mejor modelo es: Árbol de Decisión con una precisión de 0.9979757085020243
